<a href="https://colab.research.google.com/github/jjangmo91/Cervus-nippon_Anmado-Is./blob/main/GF_SG_test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# Earth Engine Python API 모듈 및 라이브러리 호출
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
from osgeo import gdal
import requests
import json
import geopandas as gpd
import os
import shutil
import psutil
from google.colab import drive
import zipfile
import glob

# Earth Engine 인증
ee.Authenticate()

# Earth Engine 초기화
ee.Initialize(project='ee-jjangmo91')

Google Drive에 저장된 데이터를 가져옵니다.

In [3]:
# Google Drive 마운트
drive.mount('/content/drive')

# 데이터 폴더 경로 설정
data_dir = '/content/drive/MyDrive/Deer/'

Mounted at /content/drive


In [7]:
# TIFF 파일 목록 가져오기
tiff_files = glob.glob(os.path.join(data_dir, '*.tif'))

# 파일명에서 날짜 추출하여 데이터프레임 생성
file_dates = []

for file_path in tiff_files:
    # 파일명 추출
    file_name = os.path.basename(file_path)

    # 파일명에서 날짜 부분 추출 (예: 'NDVI_20140101.tif'에서 '20140101' 추출)
    date_str = file_name.split('_')[1].split('.')[0]

    # 문자열을 날짜 형식으로 변환
    date = pd.to_datetime(date_str, format='%Y%m%d')

    file_dates.append({'file_path': file_path, 'date': date})

# 데이터프레임 생성
files_df = pd.DataFrame(file_dates)

# 날짜 순으로 정렬
files_df = files_df.sort_values(by='date').reset_index(drop=True)

print(files_df)

                                            file_path       date
0   /content/drive/MyDrive/Deer/20140101_20140630.tif 2014-06-30
1   /content/drive/MyDrive/Deer/20140701_20141231.tif 2014-12-31
2   /content/drive/MyDrive/Deer/20150101_20150630.tif 2015-06-30
3   /content/drive/MyDrive/Deer/20150701_20151231.tif 2015-12-31
4   /content/drive/MyDrive/Deer/20160101_20160630.tif 2016-06-30
5   /content/drive/MyDrive/Deer/20160701_20161231.tif 2016-12-31
6   /content/drive/MyDrive/Deer/20170101_20170630.tif 2017-06-30
7   /content/drive/MyDrive/Deer/20170701_20171231.tif 2017-12-31
8   /content/drive/MyDrive/Deer/20180101_20180630.tif 2018-06-30
9   /content/drive/MyDrive/Deer/20180701_20181231.tif 2018-12-31
10  /content/drive/MyDrive/Deer/20190101_20190630.tif 2019-06-30
11  /content/drive/MyDrive/Deer/20190701_20191231.tif 2019-12-31
12  /content/drive/MyDrive/Deer/20200101_20200630.tif 2020-06-30
13  /content/drive/MyDrive/Deer/20200701_20201231.tif 2020-12-31
14  /content/drive/MyDriv

In [8]:
# Anmado GeoJSON URL
url_anmado = 'https://github.com/jjangmo91/Cervus-nippon_Anmado-Is./raw/main/anmado.geojson'

# GeoJSON 파일을 GEE FeatureCollection으로 변환하는 함수
def geojson_to_feature_collection(url):
    response = requests.get(url)
    geojson = response.json()
    features = []
    for feature in geojson['features']:
        geom = ee.Geometry(feature['geometry'])
        feat = ee.Feature(geom, feature['properties'])
        features.append(feat)
    return ee.FeatureCollection(features)

# Anmado FeatureCollection 생성
anmado_fc = geojson_to_feature_collection(url_anmado)

# FRTP_CD 값 목록 (문자열 형태)
frtp_values = ['0', '1', '2', '3']

# 각 FRTP_CD 값에 대해 AOI 설정
results_anmado = []

for frtp_value in frtp_values:
    # 속성 이름 'FRTP_CD'로 필터링
    filtered_anmado_fc = anmado_fc.filter(ee.Filter.eq('FRTP_CD', frtp_value))

    # 필터링된 FeatureCollection 크기 확인
    count_anmado = filtered_anmado_fc.size().getInfo()

    print(f"FRTP_CD {frtp_value} - Anmado Feature Count: {count_anmado}")

    if count_anmado == 0:
        print(f"FRTP_CD {frtp_value}에 대한 FeatureCollection이 비어 있습니다.")
        continue  # 다음 FRTP_CD 값으로 넘어감

    # AOI 설정: 원래의 geometry 사용
    aoi_anmado = filtered_anmado_fc.geometry()

    results_anmado.append((frtp_value, aoi_anmado))

# 색상 팔레트 설정 (FRTP_CD 값별로 색상을 지정)
import matplotlib.colors as mcolors
import matplotlib.pyplot as plt

colors = plt.cm.get_cmap('tab10', len(frtp_values)).colors  # 'tab10' 색상 팔레트 사용

# RGB 색상을 16진수 문자열로 변환하는 함수
def rgb_to_hex(rgb):
    return mcolors.to_hex(rgb)

# FRTP_CD 값에 따라 색상을 매핑
color_map = {frtp_value: rgb_to_hex(colors[idx]) for idx, frtp_value in enumerate(frtp_values)}

# 지도 객체 생성 및 AOI 시각화
Map = geemap.Map(center=[35.344031, 126.028111], zoom=12)

# 각 FRTP_CD에 대해 설정한 AOI를 시각화 (Anmado)
for frtp_value, aoi in results_anmado:
    if aoi is not None:
        Map.addLayer(aoi, {'color': color_map[frtp_value]}, f'Anmado AOI FRTP_CD {frtp_value}')

# Anmado AOI 중심으로 설정
if results_anmado:
    Map.centerObject(results_anmado[0][1], 12)

# 지도 표시
Map

FRTP_CD 0 - Anmado Feature Count: 30
FRTP_CD 1 - Anmado Feature Count: 32
FRTP_CD 2 - Anmado Feature Count: 27
FRTP_CD 3 - Anmado Feature Count: 31


<ipython-input-8-aa78b2499d31>:46: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  colors = plt.cm.get_cmap('tab10', len(frtp_values)).colors  # 'tab10' 색상 팔레트 사용


TimeoutException: Requesting secret EE_PROJECT_ID timed out. Secrets can only be fetched when running from the Colab UI.